In [0]:
-- 1. CREACIÓN DE VISTA TEMPORAL: CLEANING
--    - Unifica clientes y suscripciones
--    - Imputa valores nulos usando promedios globales
--    - Filtra registros inválidos para análisis

CREATE OR REPLACE TEMP VIEW cleaning AS
SELECT
    -- Normalización de país
    COALESCE(pais, 'Desconocido') AS pais,

    -- Identificador de cliente
    cliente_id,

    -- Imputación de edad usando promedio global
    COALESCE(
        edad,
        AVG(edad) OVER ()
    ) AS edad,

    -- Imputación de pago mensual usando promedio global
    COALESCE(
        pago_mensual,
        AVG(pago_mensual) OVER ()
    ) AS pago_mensual,

    -- Información de la suscripción
    tipo_suscripcion,
    estado_suscripcion

FROM workspace.default.clientes_proyecto
INNER JOIN workspace.default.suscripciones_proyecto
USING (cliente_id)

-- Filtrado temprano de registros inválidos
WHERE tipo_suscripcion IS NOT NULL
  AND estado_suscripcion IS NOT NULL;



-- 2. VALIDACIÓN DE CALIDAD DE DATOS
--    - Conteo de valores nulos tras la limpieza

SELECT
    COUNT(*) AS nulos
FROM cleaning
WHERE pais IS NULL
   OR edad IS NULL
   OR pago_mensual IS NULL;


-- 3. ANÁLISIS AGREGADO
--    - Métricas anuales por país y tipo de suscripción
--    - Filtros de negocio y calidad

SELECT
    pais,
    tipo_suscripcion,

    -- Cálculo de ingreso anual promedio
    AVG(pago_mensual * 12) AS pago_promedio_anual,

    -- Total de clientes por grupo
    COUNT(cliente_id) AS total_clientes

FROM cleaning

-- Filtros de negocio
WHERE edad >= 21
  AND pago_mensual > 0
  AND estado_suscripcion = "Activa"

GROUP BY
    pais,
    tipo_suscripcion

-- Reglas mínimas de representatividad
HAVING
    AVG(pago_mensual * 12) > 500
    AND COUNT(cliente_id) > 10

-- Orden por mayor ingreso anual
ORDER BY
    pago_promedio_anual DESC;
